In [1]:
%load_ext autoreload
%autoreload 2

## Running experiments externally

Running experiments externally is the most common use of summit. There is a simple set of commands that allows you to do this. Let's say we are going to run the SnarBenchmark ourselves instead of in a closed-loop setup. We'll use TSEMO since this is a multiobjective problem.

In [18]:
# Import the necessary classes
from summit.benchmarks import SnarBenchmark
from summit.strategies import TSEMO2
from summit.utils.dataset import DataSet

In [19]:
# We are just going to get the domain from the SnarBenchmark
exp = SnarBenchmark()

# Instatiate TSEMO
tsemo = TSEMO2(exp.domain)

# To get the first experiments
next_experiments = tsemo.suggest_experiments(5)
next_experiments

,tau,equiv_pldn,conc_dfnb,temperature,strategy
0,1.85,4.6,0.46,39.0,LHS
1,1.55,1.4,0.22,75.0,LHS
2,0.95,3.0,0.38,57.0,LHS
3,0.65,2.2,0.30,111.0,LHS
4,1.25,3.8,0.14,93.0,LHS


You'll notice that the first set of experiments was generated by LHS or latin hypercube sampling. We'll need to go to the lab and run experiments at those conditions. 

Since it might be a while before we come back, let's save what we've done up until now. We save a file called `snar_tsemo_external.json` which contains all of the information we need to reproduce our work up to this point.  We will also produce a CSV with the suggested conditions, which we can use to record the results of our experiments. 

In [20]:
# Save TSEMO hyperparameters and internal state
tsemo.save('assets/snar_tsemo_external.json')

# Save experiments to CSV
next_experiments.to_csv('assets/snar_experiments_external_0.csv')

Normally, we would now go and run the experiments separately. We'll simulate this using the benchmark class and save the results to a CSV. 

In [21]:
# Run the experiments
results = exp.run_experiments(next_experiments)

# Save the results to a CSV file
results.to_csv('assets/snar_experiments_external_1.csv')

Once we have run the conditions, we can go ahead and load the data back in.

In [22]:
# Load in previous TSEMO hyperparameters and internal state
tsemo = TSEMO2.load('assets/snar_tsemo_external.json')

# Load in experimental results
prev_res = DataSet.read_csv('assets/snar_experiments_external_1.csv')

Now, we can run TSEMO again and ask for new suggested conditions based on the first set of experiments.

In [23]:
second_experiments = tsemo.suggest_experiments(5, prev_res=prev_res)
prev_res.append(second_experiments)
prev_res.to_csv('assets/snar_experiments_external_2.csv')
tsemo.save('assets/snar_tsemo_external_2.json')
next_experiments

,tau,equiv_pldn,conc_dfnb,temperature,strategy
0,1.85,4.6,0.46,39.0,LHS
1,1.55,1.4,0.22,75.0,LHS
2,0.95,3.0,0.38,57.0,LHS
3,0.65,2.2,0.30,111.0,LHS
4,1.25,3.8,0.14,93.0,LHS


We can repeat the the cycle of saving and loading the TSEMO paramters and next experiments until we are satisfied with the results.

## Running Closed Loop Experiments

Summit has special functionality for running closed-loop experiments.  The runner class takes a strategy and experiment. It then get experiment suggestions from the strategy and runs the experiments.

Note that in order to see the progress bar in jupyter, you'll need to install the jupyter or jupyterlab extension for ipywidgets. See [here](https://ipywidgets.readthedocs.io/en/latest/user_install.html#installing-the-jupyterlab-extension) for details.

In [2]:
# Import the necessary classes
from summit.benchmarks import Hartmann3D
from summit.strategies import SNOBFIT, NelderMead
from summit.utils.dataset import DataSet
from summit.run import Runner

In [3]:
# Establish the experiment. Here, we are using one of the single objective test functions
exp = Hartmann3D(maximize=True, constraints=False)

# Set up the strategy, passing in the experimental domain.
nm = NelderMead(exp.domain)

# Use the runner to run closed loop experiments
r = Runner(strategy=nm, experiment=exp, 
           max_iterations=5)
r.run()

> /Users/Kobi/Documents/Research/summit/summit/strategies/neldermead.py(411)inner_suggest_experiments()
    410         import ipdb; ipdb.set_trace()
--> 411         param = [sim, fsim, x_iter, red_dim, red_sim, red_fsim, rec_dim, memory]
    412 



ipdb>  sim


array([[0.5  , 0.5  , 0.5  ],
       [0.625, 0.5  , 0.5  ],
       [0.5  , 0.625, 0.5  ],
       [0.5  , 0.5  , 0.625]])


ipdb>  q


BdbQuit: 

We can also save and load the state of the runner to disk.

In [7]:
nm.prev_param[0]

[array([[0.125  , 0.66668, 0.66668],
        [0.5    , 0.5    , 0.625  ],
        [0.5    , 0.625  , 0.5    ],
        [0.5    , 0.5    , 0.5    ]]),
 array([-2.4788372 , -1.38687272, -1.06901108, -0.62802202]),
 {'xbar': array([0.375  , 0.59723, 0.59723]),
  'xr': [array([0.25   , 0.69446, 0.69446]), array([-2.50347719])],
  'xe': [array([0.     , 0.88892, 0.88892]), None],
  'xc': None,
  'xcc': None,
  'x_shrink': None},
 False,
 None,
 None,
 None,
 [array([inf, inf, inf]),
  [0.5, 0.5, 0.5],
  [0.625, 0.5, 0.5],
  [0.5, 0.625, 0.5],
  [0.5, 0.5, 0.625],
  [0.37500000000000006, 0.5833400000000001, 0.5833400000000001],
  [0.125, 0.66668, 0.66668],
  [0.25, 0.6944600000000001, 0.6944600000000001],
  [0.0, 0.88892, 0.88892]]]

In [3]:
# Save everything to a file
r.save('assets/snobfit_hartmann.json')

# Load everything from a file
Runner.load('assets/snobfit_hartmann.json')

TypeError: Object of type ndarray is not JSON serializable